In [ ]:
import time
from scipy.sparse import csr_matrix
from dolfin import *
from block import *
import fenics as fe

from xii.assembler.average_matrix import average_matrix as average_3d1d_matrix, trace_3d1d_matrix
from xii import *
from scipy.sparse import *

from ufl.corealg.traversal import traverse_unique_terminals
import dolfin as df
import ufl
import numpy as np
import matplotlib.pyplot as plt
import common_fun as fun

from dlroms.dnns import Dense, Reshape
from dlroms.minns import Local
from podminn import tanh
from np2vtk import Slab
from dlroms.cores import CPU 
import torch


#AUXILIARY FUNCTIONS

def create_submeshes(meshV):
    subdomains = MeshFunction('size_t', meshV, meshV.topology().dim())
    sub_visualize=MeshFunction('size_t', meshV, 0)

    CompiledSubDomain('x[0] < 0 + DOLFIN_EPS and x[1] < 0 + DOLFIN_EPS and x[2] < 0+ DOLFIN_EPS').mark(subdomains, 1)
    CompiledSubDomain('x[0] >= 0 - DOLFIN_EPS and x[1] < 0 + DOLFIN_EPS and x[2] >= 0 - DOLFIN_EPS').mark(subdomains, 2)
    CompiledSubDomain('x[0] < 0 + DOLFIN_EPS and x[1] < 0 + DOLFIN_EPS and x[2] >= 0- DOLFIN_EPS').mark(subdomains, 3)
    CompiledSubDomain('x[0] >= 0 - DOLFIN_EPS  and x[1] < 0 + DOLFIN_EPS and x[2] < 0+ DOLFIN_EPS').mark(subdomains, 4)
    CompiledSubDomain('x[0] < 0 + DOLFIN_EPS and x[1] >= 0 - DOLFIN_EPS and x[2] < 0 + DOLFIN_EPS').mark(subdomains, 5)
    CompiledSubDomain('x[0] >= 0 - DOLFIN_EPS and x[1] >= 0- DOLFIN_EPS and x[2] >= 0- DOLFIN_EPS').mark(subdomains, 6)
    CompiledSubDomain('x[0] < 0 + DOLFIN_EPS and x[1] >= 0- DOLFIN_EPS and x[2] >= 0- DOLFIN_EPS').mark(subdomains, 7)
    CompiledSubDomain('x[0] >= 0 - DOLFIN_EPS  and x[1] >=  0 - DOLFIN_EPS and x[2] < 0 + DOLFIN_EPS').mark(subdomains, 8)

    CompiledSubDomain('x[0] < 0 + DOLFIN_EPS and x[1] < 0 + DOLFIN_EPS and x[2] < 0 + DOLFIN_EPS').mark(sub_visualize, 1)
    CompiledSubDomain('x[0] >= 0 - DOLFIN_EPS and x[1] < 0 + DOLFIN_EPS and x[2] >= 0 - DOLFIN_EPS').mark(sub_visualize, 2)
    CompiledSubDomain('x[0] < 0 + DOLFIN_EPS and x[1] < 0+ DOLFIN_EPS and x[2] >= 0 - DOLFIN_EPS').mark(sub_visualize, 3)
    CompiledSubDomain('x[0] >= 0 - DOLFIN_EPS  and x[1] < 0+ DOLFIN_EPS and x[2] < 0 + DOLFIN_EPS').mark(sub_visualize, 4)
    CompiledSubDomain('x[0] < 0 + DOLFIN_EPS and x[1] >= 0 - DOLFIN_EPS and x[2] < 0+ DOLFIN_EPS').mark(sub_visualize, 5)
    CompiledSubDomain('x[0] >= 0 - DOLFIN_EPS and x[1] >= 0 - DOLFIN_EPS and x[2] >= 0- DOLFIN_EPS').mark(sub_visualize, 6)
    CompiledSubDomain('x[0] < 0 + DOLFIN_EPS and x[1] >= 0 - DOLFIN_EPS and x[2] >= 0 - DOLFIN_EPS').mark(sub_visualize, 7)
    CompiledSubDomain('x[0] >= 0 - DOLFIN_EPS  and x[1] >=  0 - DOLFIN_EPS and x[2] < 0+ DOLFIN_EPS').mark(sub_visualize, 8)

    submeshes=[]
    for i in range(8):
        submeshes.append(SubDomainMesh(subdomains,i+1))
        count=i+1
        File(f"sub_mesh/{count}.pvd")<< submeshes[i]

    return submeshes, subdomains, sub_visualize


def project_solution(submesh, u3d):
    W=FunctionSpace(submesh, 'CG', 1)
    u_proj=Function(W)
    u_proj=interpolate(u3d, W)
    
    return u_proj


def local_solution(submeshes, u3d):
    u_locali=[]
    u_loc_values=[]

    for i in range (8):
        u_loc=project_solution(submeshes[i],u3d)
        u_loc_val=u_loc.compute_vertex_values()
        u_locali.append(u_loc)
        u_loc_values.append(u_loc_val)
    
    return u_locali, u_loc_values


def local_global_map_dict(submeshes):
    array_dict=[]

    for i in range (8):
        mesh_curr=submeshes[i]
        dictionary=mesh_curr.parent_entity_map[0][0]
        array_dict.append(dictionary)

    return array_dict

def global_local_map_dict(array_dict):
    dict_inv=[]
    for i in range(8):
        dizionario_invertito = {valore: chiave for chiave, valore in array_dict[i].items()}
        dict_inv.append(dizionario_invertito)

    return dict_inv    

def find_index(valore, dictarray):
    c=0
    i=0
    sub_index=[]
    for dizionario in dictarray:
        if valore in dizionario.values():
            c=c+1
            sub_index.append(i)
        i=i+1     
    if c==1:
        print('presente in un unica submesh:')
    else:
        print(c)
       
    return sub_index  

def assemble_global_fun(u_loc_values,mappa_indici, dict_inv, meshV, shape_mesh):

    global_fun=np.zeros(shape_mesh, dtype=np.float32)
    for i in range (shape_mesh):
        if len(mappa_indici[i])==1:
            n_mesh=mappa_indici[i][0]
            global_fun[i]=u_loc_values[n_mesh][dict_inv[n_mesh][i]]
        else:
            num_valori=len(mappa_indici[i])
            sum=0
            for k in range (num_valori):
                n_mesh=mappa_indici[i][k]
                sum=sum + u_loc_values[n_mesh][dict_inv[n_mesh][i]]
            sum=sum/num_valori
            global_fun[i]=sum

    V=FunctionSpace(meshV, 'CG',1)
    #map=dof_to_vertex_map(V)
    #global_fun_ric=global_fun[map]
    #map=vertex_to_dof_map(V)
    #global_fun_ric=global_fun[map]
    u_ricomposta=Function(V)
    u_ricomposta.vector()[:]=global_fun

    return u_ricomposta, global_fun

def assemble_global_fun2(u_loc_fun, mappa_indici, dict_inv, meshV):
    T=FunctionSpace(meshV, 'CG',1)
    u3d_prova2=np.zeros(len(meshV.coordinates()))
    for i in range(len(meshV.coordinates())):
        sum=0
        if len(mappa_indici[i])==1:
            n_mesh=mappa_indici[i][0]
            u_val=u_loc_fun[n_mesh].compute_vertex_values()
            u3d_prova2[i]=u_val[dict_inv[n_mesh][i]]
        else:
            n_shared=mappa_indici[i]
            for k in range(len(n_shared)):
                u_val=u_loc_fun[n_shared[k]].compute_vertex_values()
                sum=sum + u_val[dict_inv[n_shared[k]][i]]
            sum=sum/len(n_shared)
            u3d_prova2[i]=sum 
    map=dof_to_vertex_map(T)
    uprova2=Function(T)
    uprova2.vector()[:]=u3d_prova2[map]
    File('fun_ricomposta.pvd')<<uprova2

    return uprova2      

def assemble_global_fun3(u_loc_fun, mappa_indici, dict_inv, meshV):
    
    T=FunctionSpace(meshV, 'CG',1)

    u_new_val=[]
    for i in range(8):
        u=u_loc_fun[i].compute_vertex_values()
        u_new_val.append(u)
    
    u3d_prova2=np.zeros(len(meshV.coordinates()))
    for i in range(len(meshV.coordinates())):
        sum=0
        if len(mappa_indici[i])==1:
            n_mesh=mappa_indici[i][0]
            #u_val=u3d_fun_new[n_mesh].compute_vertex_values()
            u_val=u_new_val[n_mesh]
            u3d_prova2[i]=u_val[array_dict_inv[n_mesh][i]]
        else:
            n_shared=mappa_indici[i]
            for k in range(len(n_shared)):
                #u_val=u3d_fun_new[n_shared[k]].compute_vertex_values()
                u_val=u_new_val[n_shared[k]]
                sum=sum + u_val[array_dict_inv[n_shared[k]][i]]
            sum=sum/len(n_shared)
            u3d_prova2[i]=sum  

    map=dof_to_vertex_map(T)
    uprova2=Function(T)
    uprova2.vector()[:]=u3d_prova2[map]
    File('fun_ricomposta.pvd')<<uprova2

    return uprova2             

    

def create_submeshfunctions(submeshes, dist):
    
    submesh_functions = []

    for submesh in submeshes:
        # Creare una nuova MeshFunction per la sottomesh
        submesh_function = MeshFunction('double', submesh, 0)
        submesh_function.set_all(0)
        
        # Mappare la MeshFunction originale sulla nuova sottomesh
        for vertex in vertices(submesh):
            parent_vertex = submesh.parent_entity_map[0][0][vertex.index()]
            #print(parent_vertex)
            submesh_function[vertex] = dist[parent_vertex]
        
        submesh_functions.append(submesh_function)

    return submesh_functions


def compute_normal_derivative(submesh, u_locale):
    V1=FunctionSpace(submesh, 'DG', 0)
    u=u_locale
    q=TestFunction(V1)
    n=FacetNormal(submesh)
    h=FacetArea(submesh)
    v_dot_n=assemble(dot(grad(u), n)*q/h*ds)
    v_dot_n_fun=Function(V1, v_dot_n)
    Q=FunctionSpace(submesh, 'CG', 1)
    normaliP1=interpolate(v_dot_n_fun, Q)

    return normaliP1


def project_sol_face(submesh, u3d):

    u_proj=project_solution(submesh, u3d)
    boundary_mesh = BoundaryMesh(submesh, 'exterior')
    V_surface = FunctionSpace(boundary_mesh, 'CG', 1)
    usurf=Function(V_surface)
    usurf=interpolate(u_proj, V_surface)
    #fun.print_u(usurf, boundary_mesh)
    coordext=V_surface.tabulate_dof_coordinates()
    values=[usurf(point) for point in coordext]
    values=np.array(values)
    
    return values

def cube_connectivity(array_dict):

    cube_connectivity={}
    for i in range(8):
        cubes=[]
        for j in range (8):
            valori_comuni = set(array_dict[i].values()).intersection(array_dict[j].values())
            if len(valori_comuni)==441:
                #print(i, 'condivide una faccia con', j)
                cubes.append(j)
        cube_connectivity[i]=cubes    

    return cube_connectivity

def normal_derivatives(submeshes, u_locali):
    
    derivate_normali=[]
    for i in range(8):
        dn=compute_normal_derivative(submeshes[i], u_locali[i])
        derivate_normali.append(dn)    

    return derivate_normali

def dn_ij(submesh, dn, cube0, array_dict, dict_inv, derivate_normali):

    dn_val=dn.compute_vertex_values()
    common_nodes={}
    for i in range(len(cube0)):
        nodi_comune=set(array_dict[0].values()).intersection(array_dict[cube0[i]].values())
        nodi_comune=list(nodi_comune)
        nodi_comune=np.array(nodi_comune)
        common_nodes[cube0[i]]=nodi_comune
    
    for i in range(len(cube0)):
        nodi_global=common_nodes[cube0[i]]
        nodi_in_i=[dict_inv[0][chiave] for chiave in nodi_global]
        nodi_in_j=[dict_inv[cube0[i]][chiave] for chiave in nodi_global]
        dn_j=derivate_normali[cube0[i]]
        dn_j_val=dn_j.compute_vertex_values()
        dn_j_val=dn_j_val[nodi_in_j]
        dn_val[nodi_in_i]=-dn_j_val    

    T=FunctionSpace(submesh, 'CG',1)
    nn=Function(T)
    map1=dof_to_vertex_map(T)
    dn_val=dn_val[map1]
    nn.vector()[:]=dn_val  

    return nn, dn_val  


def load_model():
    xfine = np.load("coordinates.npy")
    xcoarse = Slab(nx = 5, ny = 5, nz = 5, xL = 1, yL = 1, zL = 1).coordinates()
    ncoarse = len(xcoarse)
    n=40
    p=10

    phi= Dense(2402, 20, activation = tanh) + Dense(20, p, activation = None) + Reshape(p, 1)
    psi = Local(xfine, xcoarse, activation = tanh, support = 0.1) + Dense(ncoarse, n*p, activation = None) + Reshape(n, p)


    V = np.load('./V1.npy') #dati e V da passare a tensori torch
    phi.load('./phi1.npz')
    psi.load('./psi1.npz')
    psi[0].loc = list(psi[0].loc)

    return phi, psi, V


def rom_model(mu,d, phi, psi, V):
    mu=CPU.tensor(mu)
    d=CPU.tensor(d)
    d=d.reshape(1,-1)
    mu=mu.reshape(1,-1)
    #print(d.shape)
    #print(mu.shape)

    out = torch.matmul(psi(d), phi(mu)).squeeze()

    result=out.detach().cpu().numpy()@V.T

    return result

def control_criteria(u3d, u3d_old, u1d, u1d_old):
    c=np.linalg.norm(u3d_old-u3d)+np.linalg.norm(u1d_old-u1d)
    return c

def get_mesh(n, coupling_radius, path_to_1Dmesh):

    #--------------------MESHES-------------------------

    #3D mesh--------------------------------------------
    inf_point = Point(-1 - 1.1*coupling_radius, -1 - 1.1*coupling_radius, -1 - 1.1*coupling_radius)
    max_point = Point(1 + 1.1*coupling_radius, 1 + 1.1*coupling_radius, 1 + 1.1*coupling_radius)
    meshV = BoxMesh(inf_point, max_point, n, n, n)
    coord_V=meshV.coordinates()
    #--------------------------------------------------
    
    #marking 3D mesh facets



    class inlet1(SubDomain):
         def inside(self, x, on_boundary):
             return near(x[0], -1.011) and on_boundary

    class inlet2(SubDomain):
         def inside(self, x, on_boundary):
             return near(x[1], -1.011) and on_boundary

    class inlet3(SubDomain):
         def inside(self, x, on_boundary):
             return near(x[2], -1.011) and on_boundary



    class outlet1(SubDomain):
         def inside(self, x, on_boundary):
             return near(x[0], 1.011) and on_boundary

    class outlet2(SubDomain):
         def inside(self, x, on_boundary):
             return near(x[1], 1.011) and on_boundary

    class outlet3(SubDomain):
         def inside(self, x, on_boundary):
             return near(x[2], 1.011) and on_boundary



    V_markers = MeshFunction('size_t', meshV, meshV.topology().dim()-1) #facet markers

    inlet1().mark(V_markers, 111)
    inlet2().mark(V_markers, 111)
    inlet3().mark(V_markers, 111)

    outlet1().mark(V_markers, 999)
    outlet2().mark(V_markers, 999)
    outlet3().mark(V_markers, 999)
 

    #1D mesh
    
    #LOADING NETWORK XDMF 1D MESH AND ITS TAGS (each mesh function must be saved in different files)
    meshQ = Mesh()
    print(f'{path_to_1Dmesh}.xdmf')
    with XDMFFile(f'{path_to_1Dmesh}marked_mesh.xdmf') as infile:
        infile.read(meshQ)
    print(f'{path_to_1Dmesh}markers.xdmf') 
    Q_markers = MeshFunction('size_t', meshQ , 0)
  
    xdmf_file = XDMFFile(f'{path_to_1Dmesh}markers.xdmf')  
    
    xdmf_file.read(Q_markers)
    xdmf_file.close()
    
    #---------------------------------------------------------------
        
    #marking
    
    tag = 111
    tag_out=999 # per input. 999 per output.
    
    ds1d = Measure('ds', domain=meshQ)
    ds1d = ds(subdomain_data=Q_markers)

  
    return meshV, meshQ, Q_markers, ds1d, V_markers


def get_system(meshV, meshQ, ubc3 , ubc1, ds1d,coupling_radius=0.001):
    """A, b, W, bcs"""

    Q = FunctionSpace(meshQ, 'CG', 1)
    Q1 = FunctionSpace(meshV, 'CG', 1)

    dofmap = Q1.dofmap()
    V_DOF = dofmap.global_dimension()
    print("ecco 3D dofs", V_DOF)

    W = [Q1, Q]

    u, p = map(TrialFunction, W)
    v, q = map(TestFunction, W)
   
    #computing average operator------------------------------------------
   
    # Average (coupling_radius > 0) or trace (coupling_radius = 0)
    if coupling_radius > 0:
        # Averaging surface
        cylinder = Circle(radius=coupling_radius, degree=10)
        Ru, Rv = Average(u, meshQ, cylinder), Average(v, meshQ, cylinder)
        C = average_3d1d_matrix(Q1, Q, cylinder)
    else:
        Ru, Rv = Average(u, meshQ, None), Average(v, meshQ, None)
        C = trace_3d1d_matrix(Q1, Q, meshQ)

    #--------------------------------------------------------------------         
        
    # Line integral
    dx_ = Measure('dx', domain=meshQ)

    #-------------------------------------------------------------------

    # We're building a 2x2 problem with 3D reaction---------------------
    
    R=coupling_radius

    # mm - bar
    K1=Constant(7.6*10**-11)
    K=Constant(1.03*10**-6/(8*5*10**-4)* 3.14 * R**4*10**5)
    J=Constant(6.28*10**-12)
    B=Constant(10**-6)
    P=Constant((3300-666)*10**-5)
    lamb=1
    lamb=Constant(-lamb*np.pi*R**2)
    

    tag_in=111
    tag_out=999
    a = block_form(W, 2)
    a[0][0] = K1 * inner(grad(u), grad(v)) * dx + B * inner(u, v) * ds
    #a[0][0] = K1 * inner(grad(u), grad(v)) * dx + B * inner(u, v) * ds3d(111, domain=meshV) +  B * inner(u, v) * ds3d(999, domain=meshV)
    a[1][1] = K * inner(grad(p), grad(q)) * dx -lamb*inner(p,q)*ds1d(tag_in,domain=meshQ)-lamb*inner(p,q)*ds1d(tag_out,domain=meshQ)
    
    m = block_form(W, 2)
    m[0][0] =  J* inner(Ru, Rv) * dx_
    m[0][1] = -J* inner(p, Rv) * dx_
    m[1][0] = -J*inner(q, Ru) * dx_
    m[1][1] = J*inner(p, q) * dx_

    L = block_form(W, 1)
    #L[0] =  - J*inner(P,Rv) * dx_ + B*inner(p3d_exact,v)*ds3d(111,domain=meshV)
    #L[0] =  - J*inner(P,Rv) * dx_ + B*inner(ubc,v)*ds3d(111, domain=meshV) + B*inner(ubc,v)*ds3d(999, domain=meshV)
    L[0] =  - J*inner(P,Rv) * dx_ + B*inner(ubc3,v)*ds

    #L[1] =  + J*inner(P,q)*dx_ - lamb*inner(ubc1,q)*ds1d(tag_in,domain=meshQ)- lamb*inner(ubc1,q)*ds1d(tag_out,domain=meshQ)
    L[1] =  + J*inner(P,q)*dx_ - lamb*inner(Constant(0.07),q)*ds1d(tag_in,domain=meshQ)- lamb*inner(Constant(0.05),q)*ds1d(tag_out,domain=meshQ)

    print(np.shape(a))
    print(np.shape(m))
    print(np.shape(L))

    AD, M, b = map(ii_assemble, (a, m, L))
    AAA = ii_assemble(a + m)
    
    C = csr_matrix(C.getValuesCSR()[::-1], shape=C.size)


    return (AD, M), b, W, C, V_DOF, AAA


def getGraphDist(mesh1D, mesh3D, u_dict):
   
    from closest_point_in_mesh import  closest_point_in_mesh
    '''Getting the  distance function from a mesh'''
    
    dist = MeshFunction("double", mesh3D,0)
    coord1d=mesh1D.coordinates()
    
    tree=mesh1D.bounding_box_tree()
    tree.build(mesh1D,1)
    
    for i, cord in enumerate(mesh3D.coordinates()):
    
        close_p = closest_point_in_mesh(cord, mesh1D)
        p=tuple(close_p)
        
        if p in u_dict:
            u=u_dict[p]
            val=(1-np.linalg.norm(cord-close_p))*u
        else:
            point=Point(close_p[0] , close_p[1], close_p[2])
            cell_ind,_=tree.compute_closest_entity(point)
            m=MeshEntity(mesh1D, 1,cell_ind)
            line=m.entities(0)
            p1=tuple(coord1d[line[0]])
            u1=u_dict[p1]
            p2=tuple(coord1d[line[1]])
            u2=u_dict[p2]
            d1=np.linalg.norm(close_p-coord1d[line[0]])
            d2=np.linalg.norm(close_p-coord1d[line[1]])
            d=d1+d2
            u=u1*d1/d+u2*d2/d
            val=(1-np.linalg.norm(cord-close_p))*u
            
        dist.array()[i] = val

    return dist
       

def solve_1d(meshV, meshQ, ds1d, u_3d):

    from petsc4py import PETSc

    Q=FunctionSpace(meshQ, 'CG',1)
    
    u_1d=Function(Q)
    u_1d=interpolate(u_3d, Q)

    radius=1e-2

    (AD, M), b, W, C, V_DOF, AAA = get_system(meshV, meshQ, u_3d, u_1d, ds1d,coupling_radius=radius)
    b_PET = ii_convert(b[1])
    shape = b_PET.size()  
    
    A=AD[1][1]+M[1][1]
    A_PET=ii_convert(A).mat()

    b2=M[1][0]
    M_PET=ii_convert(b2).mat()

    u3d=u_3d.compute_vertex_values()
    u3d=u3d.reshape(1,-1)
    u3d_PET = PETSc.Vec().createWithArray(u3d)

    comm = PETSc.COMM_WORLD

    n=shape
    y = PETSc.Vec().create(comm)
    y.setSizes(n)
    y.setFromOptions()

    M_PET.mult(u3d_PET, y)
    b_PET = PETSc.Vec().createWithArray(b_PET)

    b_final=b_PET-y
    u = np.zeros((shape))
    tol   = 1e-15
    u_PET = PETSc.Vec().createWithArray(u)

    solver = PETSc.KSP().create()
    solver.setOperators(A_PET)
    solver.setType(PETSc.KSP.Type.GMRES)  # <--Choose the solver type
    solver.setFromOptions()  # <--Allow setting options from the command line or a file
    solver.setTolerances(rtol=tol)
    solver.setPCSide(1)
    solver.view()

    # Set preconditioner
    pc = solver.getPC()
    pc.setType(PETSc.PC.Type.HYPRE)
    pc.setHYPREType("boomeramg")  # Set to use AMG

    solver.solve(b_PET, u_PET)
    '''
    # print
    print ('iterations = ',               solver.getIterationNumber())
    print ('residual = ', '{:.2e}'.format(solver.getResidualNorm()))#  %.2E
    print ('converge reason = ',          solver.getConvergedReason())
    print ('residuals at each iter = ',   solver.getConvergenceHistory())
    print ('precond type', pc.getType())
    # Print solver information
    num_iter = solver.getIterationNumber()
    print(f"Number of iterations: {num_iter}")

    print("\n------------------ System setup and assembly time: ", "\n")
    '''
    u_npy=np.array(u_PET.getArray())
    u1d=Function(W[1])
    u1d.vector()[:]=u_npy

    coord1d=W[1].tabulate_dof_coordinates()
    u_dict={}
    for i in range(len(coord1d)):
        key=tuple(coord1d[i])
        u_dict[key]=u_npy[i]

    distanza=getGraphDist(meshQ, meshV,u_dict)   

    return u1d, distanza


def boundary_cond3d(min, max, meshV,seed):
    
    
    coupling_radius=1e-2
    face_values, cube_vertices, cube_faces=fun.generate_face_values(min,max,coupling_radius, seed)
    vertex_values_dict=fun.compute_vertex_values(cube_vertices, cube_faces, face_values)
    u_d=fun.compute_u(meshV, coupling_radius, vertex_values_dict)   
    #fun.print_u(u_d, meshV)

    return u_d

def val_facce(min, max, meshV, seed):
    coupling_radius=1e-2
    face_values, cube_vertices, cube_faces=fun.generate_face_values(min,max,coupling_radius, seed)
    val_list=list(face_values.values())
    val=np.array(val_list)  
    return val



def boundary_cond1d(min, max, meshQ, meshV, seed):

    u_3d=boundary_cond3d(min,max,meshV,seed)
    Q=FunctionSpace(meshQ, 'CG',1)
    
    u_1d=Function(Q)
    u_1d=interpolate(u_3d, Q)

    return u_1d

def smooth(u3d, Q3d, Tcoarse, alpha):
    ucoarse=Function(Tcoarse)
    ucoarse=interpolate(u3d, Tcoarse)
    ufine=Function(Q3d)
    ufine=interpolate(ucoarse, Q3d)
    u_final=Function(Q3d)
    u_final.vector()[:]=(1-alpha)*u3d.vector()[:]+alpha*ufine.vector()[:]
    return u_final
    

In [ ]:
n=40
radius=1e-2
path_to_1Dmesh = f'888_mesh/888_'
#path_to_1Dmesh = f'net2/2_'

meshV, meshQ, markers,ds1d, V_markers= get_mesh(n, radius, path_to_1Dmesh)

submeshes, submesh_fun, sub_vis=create_submeshes(meshV)

Q3d=FunctionSpace(meshV, 'CG',1)
Q1d=FunctionSpace(meshQ, 'CG',1)

shape3d=len(meshV.coordinates())
shape1d=len(meshQ.coordinates())

In [ ]:
#COARSER MESH
nn=10
coupling_radius=1e-2
inf_point = Point(-1 - 1.1*coupling_radius, -1 - 1.1*coupling_radius, -1 - 1.1*coupling_radius)
max_point = Point(1 + 1.1*coupling_radius, 1 + 1.1*coupling_radius, 1 + 1.1*coupling_radius)
meshcoarse = BoxMesh(inf_point, max_point, nn, nn, nn)
T_coarse=FunctionSpace(meshcoarse, 'CG',1)

In [ ]:
nn=20
coupling_radius=1e-2
inf_point = Point(-1 - 1.1*coupling_radius, -1 - 1.1*coupling_radius, -1 - 1.1*coupling_radius)
max_point = Point(1 + 1.1*coupling_radius, 1 + 1.1*coupling_radius, 1 + 1.1*coupling_radius)
meshcoarse1 = BoxMesh(inf_point, max_point, nn, nn, nn)
T_coarse1=FunctionSpace(meshcoarse1, 'CG',1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Traccia i punti della mesh con i valori della funzione u
scatter=ax.scatter(meshV.coordinates()[:, 0], meshV.coordinates()[:, 1], meshV.coordinates()[:, 2])
#colorbar = fig.colorbar(scatter)

# Etichette degli assi
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

In [ ]:
def initialize_1d(meshQ, Q_markers):
    Q=FunctionSpace(meshQ, 'CG',1)
    ind=Function(Q)

    ind.vector()[:] = 0.06
    bc1 = DirichletBC(Q, Constant(0.07), Q_markers, 111)
    bc2=DirichletBC(Q, Constant(0.05), Q_markers, 999)
    bc1.apply(ind.vector())
    bc2.apply(ind.vector())

    return ind

In [ ]:
def initialize_3d(meshV, V_markers):
    V=FunctionSpace(meshV, 'CG',1)
    u=Function(V)
    u.vector()[:] = 0.04
    bc1 = DirichletBC(V, Constant(0.03), V_markers, 111)
    bc2=DirichletBC(V, Constant(0.03), V_markers, 999)
    bc1.apply(u.vector())
    bc2.apply(u.vector())

    return u

In [ ]:

u1d_old=initialize_1d(meshQ, markers)
u3d_old=initialize_3d(meshV, V_markers)
File('u_init3d.pvd')<<u3d_old
File('u_init1d.pvd')<<u1d_old
cc=10
tol=1e-10

import pickle

# LOAD DICT
with open('dizionario.pkl', 'rb') as f:
    mappa_indici = pickle.load(f)

array_dict=local_global_map_dict(submeshes)
array_dict_inv=global_local_map_dict(array_dict)
cubes_connectivity=cube_connectivity(array_dict)

# LOAD ROM
phi, psi, V=load_model()

coord1d=Q1d.tabulate_dof_coordinates()
u_dict={}
u1d_npy=u1d_old.compute_vertex_values()
for i in range(shape1d):
    key=tuple(coord1d[i])
    u_dict[key]=u1d_npy[i]

In [ ]:
distanza=getGraphDist(meshQ, meshV,u_dict)


file = XDMFFile(MPI.comm_world, f"dist_markers.xdmf")
file.parameters["flush_output"] = True
# Write the mesh and mesh function to the XDMF file
file.write(distanza)
# Close the XDMFFile
file.close()

In [ ]:
# SOL FOM USED TO ANALYZE THE ERROR
uref=np.load('ufom8.npy')

In [ ]:
K1=7.6*10**-11
B=10**-9

In [ ]:
u3d_fun_new.clear()
u3d_val_new.clear()

In [ ]:
u3d_fun_new=[]
u3d_val_new=[]
ccc=[]
cref=[]
R=FunctionSpace(meshV, 'CG', 1)
urefe=Function(R)
urefe.vector()[:]=uref
ccref=10

In [ ]:
kk=0
while( kk <101 or ccref <1e-5):

    submesh_functions=create_submeshfunctions(submeshes, distanza)
    u_locali, u_loc_values=local_solution(submeshes, u3d_old)
    derivate_normali=normal_derivatives(submeshes, u_locali)


    u3d_fun_new.clear()
    u3d_val_new.clear()


    for i in range (8):

    
        File(f'u_loc_{i}.pvd')<<u_locali[i]
        curr_mesh=submeshes[i]
        curr_cube_conn=cubes_connectivity[i]
        d=submesh_functions[i]
        mu_1=project_sol_face(curr_mesh, u_locali[i])
        
        
        Q=FunctionSpace(curr_mesh, 'CG',1)
        shape=len(Q.tabulate_dof_coordinates())
        zeros=np.zeros(shape)
        dn=Function(Q)
        dn.vector()[:]=zeros
        dn_fun, dn_val=dn_ij(curr_mesh,dn,curr_cube_conn, array_dict, array_dict_inv, derivate_normali)
        
        mu_2=project_sol_face(curr_mesh, dn_fun)
        
        mu=(B*mu_1-K1*mu_2)/B

        k=d.array()
        
        u3d_loc=rom_model(mu, k, phi, psi, V)
        
        Q=FunctionSpace(curr_mesh, 'CG',1)
        u3d_fun_loc=Function(Q)
        u3d_fun_loc.vector()[:]=u3d_loc
        File(f'u_rom_{i}.pvd')<<u3d_fun_loc

        u3d_fun_new.append(u3d_fun_loc)
        u3d_val_new.append(u3d_fun_loc.compute_vertex_values())
        

    if kk < 25:
        u3d_global_fun=assemble_global_fun3(u3d_fun_new, mappa_indici, array_dict_inv, meshV) 
        u3d_global_fun2=smooth(u3d_global_fun,Q3d, T_coarse, 1)
    else:
        u3d_global_fun=assemble_global_fun3(u3d_fun_new, mappa_indici, array_dict_inv, meshV) 
        u3d_global_fun2=smooth(u3d_global_fun,Q3d, T_coarse, 1)
        
    if kk % 10 == 0:
        u1d_new, distanza=solve_1d(meshV, meshQ, ds1d, u3d_global_fun)
        u1d_old=u1d_new

    cc3=np.linalg.norm(u3d_global_fun2.compute_vertex_values()-u3d_old.compute_vertex_values())/np.linalg.norm(u3d_old.compute_vertex_values())
    ccref=np.linalg.norm(u3d_global_fun2.compute_vertex_values()-urefe.compute_vertex_values())/np.linalg.norm(urefe.compute_vertex_values())
    print('control criteria 3d:', cc3)
    print('-----------------it-----------------', kk)
    u3d_old=u3d_global_fun2
    ccc.append(cc3)
    cref.append(ccref)
    
    #FOR SAVING
    if kk%10==0:   
        File(f'prova3d_{kk}.pvd')<<u3d_global_fun2
        File(f'prova1d_{kk}.pvd')<< u1d_new
    kk=kk+1

In [ ]:
#TO PLOT

import matplotlib.pyplot as plt

# Esempio di lista di valori

# Indici per le ascisse

indici = np.arange(1, len(cref) + 1)
# Configurazione dello stile simile a Matlab
#plt.style.use('seaborn-darkgrid')

# Creazione del grafico
plt.figure(figsize=(8, 6))
#
plt.plot(indici, cref, 'x-', markersize=1, linewidth=1, label='beta=1e-8')

# Aggiunta di etichette agli assi
plt.xlabel('Iterations', fontsize=14)

plt.ylabel('|| DD-ROM -FOM ||/ || FOM ||', fontsize=14)


# Titolo del grafico
#plt.title('Convergence analysis', fontsize=16)

# Aggiunta di una griglia
#plt.grid(True)

# Aggiunta di una legenda
plt.legend()

# Miglioramento della visualizzazione degli assi
#plt.xticks(indici)
#plt.yticks(np.arange(min(ccc), max(ccc), 2))

# Mostra il grafico
plt.show()



In [ ]:
File('3dnonsmooth.pvd')<<u3d_global_fun
File('u3dsmooth.pvd')<<u3d_global_fun2